In [34]:
# var inspector (install in anaconda prompt):
#   conda install -c conda-forge jupyterlab-variableinspector

In [35]:
# %conda install tensorflow
# %conda install tensorflow-gpu
# %conda install numpy
# %conda install pandas
# %conda install -c conda-forge matplotlib
# %conda install keras
# %conda install Pillow

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import glob
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14590995954416074513,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5767102464
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5113375404838736839
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"]

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [4]:
# image size: 1080*1920(9:16)
INPUT_HEIGHT = 144
INPUT_WIDTH = 256
INPUT_CHANNEL = 3
INPUT_SHAPE = (INPUT_HEIGHT,INPUT_WIDTH,INPUT_CHANNEL)

OUTPUT_HEIGHT = INPUT_HEIGHT
OUTPUT_WIDTH = INPUT_WIDTH
OUTPUT_CHANNEL = INPUT_CHANNEL
OUTPUT_SHAPE = INPUT_SHAPE

In [5]:
import glob
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# FIXME: tf.data.Dataset maybe faster
# e.g. dir_name = 'data/stimuli.out/'
def x_from_dir(dir_name):
    x_paths = glob.glob(dir_name + '*.jpg', recursive=False)[:2000]
    idx = 0
    
    # initialize
    first_img = load_img(x_paths[0], target_size=(INPUT_HEIGHT, INPUT_WIDTH))
    x = img_to_array(first_img).reshape(1,INPUT_HEIGHT,INPUT_WIDTH,INPUT_CHANNEL)
    # remaining
    for path in x_paths[1:]:
        img = load_img(path, target_size=(INPUT_HEIGHT, INPUT_WIDTH))
        arr = img_to_array(img).reshape(1,INPUT_HEIGHT,INPUT_WIDTH, INPUT_CHANNEL)
        x = np.append(x, arr, axis=0)
        if idx%250 == 0:
            print(f'now loading {idx}-th image')
        idx = idx + 1

    return x / 255

# FIXME: tf.data.Dataset maybe faster
# e.g. dir_name = 'data/heatmap/'
def y_from_dir(dir_name):
    y_paths = glob.glob(dir_name + '*_heatmap.png', recursive=False)
    idx = 0
    
    # initialize
    first_img = load_img(y_paths[0], color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
    y = img_to_array(first_img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
    # remaining
    for path in y_paths[1:]:
        img = load_img(path, color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
        arr = img_to_array(img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
        y = np.append(y, arr, axis=0)
        if idx%100 == 0:
            print(f'now loading {idx}-th image')
        idx = idx + 1

    return y / 256

def salicon_y_from_dir(dir_name):
    y_paths = glob.glob(dir_name + '*.png', recursive=False)[:2000]
    idx = 0
    
    # initialize
    first_img = load_img(y_paths[0], color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
    y = img_to_array(first_img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
    # remaining
    for path in y_paths[1:]:
        img = load_img(path, color_mode='grayscale', target_size=(OUTPUT_HEIGHT, OUTPUT_WIDTH))
        arr = img_to_array(img).reshape(1,OUTPUT_HEIGHT,OUTPUT_WIDTH,1)
        y = np.append(y, arr, axis=0)
        if idx%250 == 0:
            print(f'now loading {idx}-th image')
        idx = idx + 1

    return y / 256

In [44]:
# https://jpn.pioneer/ja/strengths/crdl/rd/pdf/2020-1.pdf
from tensorflow.keras import datasets, layers, models

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=INPUT_SHAPE, padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(1, (3, 3), activation='relu', padding='same'))

model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 144, 256, 64)      1792      
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 144, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 72, 128, 64)       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 72, 128, 128)      73856     
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 72, 128, 128)      147584    
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 72, 128, 128)      147584    
_________________________________________________________________
up_sampling2d_7 (UpSampling2 (None, 144, 256, 128)    

In [62]:
# https://jpn.pioneer/ja/strengths/crdl/rd/pdf/2020-1.pdf
# series 用
from tensorflow.keras import datasets, layers, models

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=INPUT_SHAPE, padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(1, (3, 3), activation='relu', padding='same'))

model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_88 (Conv2D)           (None, 144, 256, 64)      1792      
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 144, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 72, 128, 64)       0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 72, 128, 64)       36928     
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 72, 128, 96)       55392     
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 72, 128, 96)       83040     
_________________________________________________________________
up_sampling2d_11 (UpSampling (None, 144, 256, 96)    

In [118]:
x_dir = 'data/stimuli.out/'
x = x_from_dir(x_dir)
y_dir = 'data/heatmap/'
y = y_from_dir(y_dir)

now loading 0-th image
now loading 250-th image
now loading 0-th image
now loading 100-th image
now loading 200-th image


## 1st split

In [107]:
# slice all data
TRAIN_SIZE = 240
x_train = x[:TRAIN_SIZE]
x_test = x[TRAIN_SIZE:]
y_train = y[:TRAIN_SIZE]
y_test = y[TRAIN_SIZE:]
model.fit(x_train, y_train, batch_size=10, epochs=10)

Epoch 1/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0656 - accuracy: 0.9218
Epoch 2/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0652 - accuracy: 0.9218
Epoch 3/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0650 - accuracy: 0.9218
Epoch 4/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0648 - accuracy: 0.9218
Epoch 5/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0647 - accuracy: 0.9218
Epoch 6/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0646 - accuracy: 0.9218
Epoch 7/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0644 - accuracy: 0.9218
Epoch 8/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0642 - accuracy: 0.9218
Epoch 9/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0642 - accuracy: 0.9218
Epoch 10/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0641 - accuracy: 0.9218


In [108]:
from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255
    
    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

In [109]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 22.972861528396606


## 2nd split

In [110]:
# slice all data
x_train = np.concatenate([ x[:180], x[240:] ])
x_test = x[180:240]
y_train = np.concatenate([ y[:180], y[240:] ])
y_test = y[180:240]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    # img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255
    
    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

Epoch 1/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0658 - accuracy: 0.9221
Epoch 2/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0652 - accuracy: 0.9221
Epoch 3/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0651 - accuracy: 0.9221
Epoch 4/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0650 - accuracy: 0.9221
Epoch 5/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0645 - accuracy: 0.9221
Epoch 6/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0648 - accuracy: 0.9221
Epoch 7/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0646 - accuracy: 0.9221
Epoch 8/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0648 - accuracy: 0.9221
Epoch 9/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0648 - accuracy: 0.9221
Epoch 10/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0645 - accuracy: 0.9221


In [111]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 23.83895425796509


## 3rd split

In [112]:
# slice all data
x_train = np.concatenate([ x[:120], x[180:] ])
x_test = x[120:180]
y_train = np.concatenate([ y[:120], y[180:] ])
y_test = y[120:180]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    # img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255

    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

Epoch 1/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0638 - accuracy: 0.9231
Epoch 2/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0634 - accuracy: 0.9231
Epoch 3/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0633 - accuracy: 0.9231
Epoch 4/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0632 - accuracy: 0.9231
Epoch 5/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0629 - accuracy: 0.9231
Epoch 6/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0629 - accuracy: 0.9231
Epoch 7/10
24/24 [==============================] - 2s 70ms/step - loss: 0.0626 - accuracy: 0.9231
Epoch 8/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0623 - accuracy: 0.9231
Epoch 9/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0624 - accuracy: 0.9231
Epoch 10/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0627 - accuracy: 0.9231


In [113]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 17.026801109313965


## 4th split

In [114]:
# slice all data
x_train = np.concatenate([ x[:60], x[120:] ])
x_test = x[60:120]
y_train = np.concatenate([ y[:60], y[120:] ])
y_test = y[60:120]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))
    # img.save('data/result/' + str(i) + '.png')

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255

    # fig, ax = plt.subplots()
    # ax.imshow(heat_sq)
    # fig, ax = plt.subplots()
    # ax.imshow(true)
    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

Epoch 1/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0633 - accuracy: 0.9220
Epoch 2/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0628 - accuracy: 0.9220
Epoch 3/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0630 - accuracy: 0.9220
Epoch 4/10
24/24 [==============================] - 2s 72ms/step - loss: 0.0624 - accuracy: 0.9219
Epoch 5/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0624 - accuracy: 0.9220
Epoch 6/10
24/24 [==============================] - 2s 72ms/step - loss: 0.0618 - accuracy: 0.9220
Epoch 7/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0620 - accuracy: 0.9220
Epoch 8/10
24/24 [==============================] - 2s 72ms/step - loss: 0.0618 - accuracy: 0.9220
Epoch 9/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0614 - accuracy: 0.9220
Epoch 10/10
24/24 [==============================] - 2s 71ms/step - loss: 0.0614 - accuracy: 0.9220


In [115]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 17.47025508880615


## 5th split

In [116]:
# slice all data
x_train = x[60:]
x_test = x[:60]
y_train = y[60:]
y_test = y[:60]
model.fit(x_train, y_train, batch_size=10, epochs=10)

from tensorflow.keras.utils import array_to_img
from matplotlib import pyplot as plt

y_pred = model.predict(x_test)

# save predicted imgs
i = 0
l2 = 0
for inst in y_pred:
    heat = (inst / np.max(inst)) ** 5 * 255
    heat = np.floor(heat).astype(int)
    img = array_to_img(heat).resize((1920, 1080))

    heat_norm = (inst / np.max(inst)) ** 5
    heat_sq = img_to_array( array_to_img(heat_norm).resize((200, 200)) ) / 255
    true = img_to_array( array_to_img(y_test[i]).resize((200, 200)) ) / 255

    l2 += np.linalg.norm(heat_sq-true)
    
    i += 1

Epoch 1/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0627 - accuracy: 0.9210
Epoch 2/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0626 - accuracy: 0.9210
Epoch 3/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0622 - accuracy: 0.9210
Epoch 4/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0617 - accuracy: 0.9210
Epoch 5/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0619 - accuracy: 0.9210
Epoch 6/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0659 - accuracy: 0.9210
Epoch 7/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0665 - accuracy: 0.9210
Epoch 8/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0664 - accuracy: 0.9210
Epoch 9/10
24/24 [==============================] - 2s 75ms/step - loss: 0.0662 - accuracy: 0.9210
Epoch 10/10
24/24 [==============================] - 2s 74ms/step - loss: 0.0659 - accuracy: 0.9210


In [117]:
print(f"mean l2 distance: {l2 / len(y_pred)}")

mean l2 distance: 23.232016881306965


## base

In [6]:
import pandas as pd
import numpy as np

np.mean(pd.read_csv('data/base_evals.csv', header=None)[1])

63.55988916666665